# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [83]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.data.datapath import DataPath
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from azureml.core import Workspace, Dataset
from azureml.train.automl.utilities import get_primary_metrics
from azureml.widgets import RunDetails
from azureml.core.run import Run
from azureml.train.automl.run import AutoMLRun

from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.model import Model
from azureml.core.webservice import AciWebservice

from azureml.core.authentication import InteractiveLoginAuthentication
import requests

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform, normal
from azureml.core import Environment, ScriptRunConfig
import os
from azureml.train.hyperdrive import BayesianParameterSampling
from azureml.train.hyperdrive import uniform, choice
from azureml.train.hyperdrive import HyperDriveRun

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.49.0


In [84]:
interactive_auth = InteractiveLoginAuthentication(tenant_id="660b3398-b80e-49d2-bc5b-ac1dc93b5254")
ws = Workspace(subscription_id="9b72f9e6-56c5-4c16-991b-19c652994860",
               resource_group="aml-quickstarts-232431",
               workspace_name="quick-starts-ws-232431",
               auth=interactive_auth)
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-232431
aml-quickstarts-232431
westeurope
9b72f9e6-56c5-4c16-991b-19c652994860


In [3]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

UserErrorException: UserErrorException:
	Message: You are currently logged-in to 660b3398-b80e-49d2-bc5b-ac1dc93b5254 tenant. You don't have access to b968fb36-f06a-4c76-a15f-afab68ae7667 subscription, please check if it is in this tenant. All the subscriptions that you have access to in this tenant are = 
 [SubscriptionInfo(subscription_name='Udacity CloudLabs Sub - 01', subscription_id='510b94ba-e453-4417-988b-fbdc37b55ca7')]. 
 Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "You are currently logged-in to 660b3398-b80e-49d2-bc5b-ac1dc93b5254 tenant. You don't have access to b968fb36-f06a-4c76-a15f-afab68ae7667 subscription, please check if it is in this tenant. All the subscriptions that you have access to in this tenant are = \n [SubscriptionInfo(subscription_name='Udacity CloudLabs Sub - 01', subscription_id='510b94ba-e453-4417-988

In [85]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "Project-3-Compute"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS3_v2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 0, timeout_in_minutes = 10)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [86]:
experiment_name = 'Project_3_Experiment_Hyperdrive'
experiment=Experiment(ws, experiment_name)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [87]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
param_sampling = BayesianParameterSampling(
    {
    "max_iter": choice(range(200, 3000, 100)),
    "activation": choice(['identity', 'logistic', 'relu', 'tanh'])
    }
)

#TODO: Create your estimator and hyperdrive config
if "training" not in os.listdir():
    os.mkdir("./training")
    
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')


estimator = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=compute_target,
                      environment=sklearn_env)

hyperdrive_run_config = HyperDriveConfig(run_config=estimator,
                                     hyperparameter_sampling=param_sampling,
                                     policy=early_termination_policy,
                                     primary_metric_name='normalized_root_mean_squared_error',
                                     primary_metric_goal=PrimaryMetricGoal.MINIMIZE,
                                     max_total_runs=40,
                                     max_concurrent_runs=4)
hyperdrive_run = experiment.submit(config=hyperdrive_run_config)

In [88]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(config=hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [89]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [90]:
hyperdrive_run.wait_for_completion()

{'runId': 'HD_8e34cc7a-f03a-43b6-a5a0-78776ca66757',
 'target': 'Project-3-Compute',
 'status': 'Completed',
 'startTimeUtc': '2023-05-01T21:39:49.981118Z',
 'endTimeUtc': '2023-05-01T22:00:55.993443Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"normalized_root_mean_squared_error","goal":"minimize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '186b10c7-901d-4682-9ce5-3a5fc15313fc',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1035-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.49.0',
  'space_size': '112',
  'score': '0.11489942774374742',
  'best_child_run_id': 'HD_8e34cc7a-f03a-43b6-a5a0-78776ca66757_15',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_8e34cc7a-f03a-43b6-a5a0-78776ca66757_15'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': Non

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [91]:
#TODO: Save the best model
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print('Best Run Id: ', best_run.id)
print('\n\normalized_root_mean_squared_error:', best_run_metrics['normalized_root_mean_squared_error'])

Best Run Id:  HD_8e34cc7a-f03a-43b6-a5a0-78776ca66757_15


ormalized_root_mean_squared_error: 0.11489942774374742


In [92]:
best_run_metrics

{'Maximum Iterations:': 2200.0,
 'Activation Function:': 'identity',
 'normalized_root_mean_squared_error': 0.11489942774374742}

In [44]:
hyperdrive_run = HyperDriveRun(experiment, [run for run in experiment.get_runs()][0].id)

In [93]:
print(best_run.get_file_names())

['azureml-logs/hyperdrive.txt', 'outputs/model.joblib', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/metrics_capability/metrics-capability.log', 'system_logs/snapshot_capability/snapshot-capability.log', 'user_logs/std_log.txt']


In [69]:
type(best_run)

azureml.core.run.Run

In [95]:
model = best_run.register_model(model_name='Behavioral_Model_Hyperdrive', model_path='outputs/model.joblib')

In [96]:
model

Model(workspace=Workspace.create(name='quick-starts-ws-232431', subscription_id='9b72f9e6-56c5-4c16-991b-19c652994860', resource_group='aml-quickstarts-232431'), name=Behavioral_Model_Hyperdrive, id=Behavioral_Model_Hyperdrive:1, version=1, tags={}, properties={})

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.



# Cleanup

In [ ]:
compute_target.delete()